# Cython

## Loading the extension

IPtyhon notebook import Cython code. This extension can be loaded using the `pyximport` as follows:

In [17]:
import pyximport
pyximport.install(reload_support=True)
import fib_cython

## Using the Cython module

foo is the already compiling module function. 
Using foo module as follows.

In [22]:
fib_cython.fib(2)

1

Writing a normal function as follows.

In [ ]:
def fib(n):
    a, b = 0, 1
    for i in range(n):
        a, b = a + b, a
    return a

In [20]:
fib(2)

1